Breast Cancer Classification

1-Loading Dataset

In [95]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier,LocalOutlierFactor
from sklearn.decomposition import PCA

file="breast_cancer.csv"
data=pd.read_csv(file)
df=pd.DataFrame(data)
  


Pre-processing

In [93]:
df.head()
df=df.drop('Unnamed: 32',axis=1)
df=pd.get_dummies(df,'diagnosis',drop_first=True)
df.columns
y=df['diagnosis_M']
x=df.drop('diagnosis_M',axis=1)
y.head()

0    True
1    True
2    True
3    True
4    True
Name: diagnosis_M, dtype: bool

2-Exploring data

In [ ]:
corr_matrix=df.corr()
sns.clustermap(corr_matrix,annot=True)
plt.tight_layout()

In [ ]:
threshold=0.8
filtered_corr = corr_matrix[(np.abs(corr_matrix) >= threshold) & (corr_matrix != 1.0)]
sns.heatmap(filtered_corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.show()

3-Defining Outliers

In [ ]:
y=df['diagnosis_M'] 
x=df.drop('diagnosis_M',axis=1) 
columns=x.columns.tolist() 
clf=LocalOutlierFactor() 
y_pred=clf.fit_predict(x) 
X_scores=clf.negative_outlier_factor_ 
outlier_score=pd.DataFrame() 
outlier_score['scores']=X_scores 
threshold=-2.2 
ilter=outlier_score['scores']<threshold 
outlier_index=outlier_score[filter].index.to_list() 
plt.figure(figsize=(12, 8)) 
plt.scatter(x.iloc[outlier_index, 0], x.iloc[outlier_index, 1], color="blue", s=50, label="Aykırılar") 
plt.scatter(x.iloc[:, 0], x.iloc[:, 1], color="k", s=3, label="Veri Noktaları") 
plt.legend() 
plt.show()

x=x.drop(outlier_index)
y=y.drop(outlier_index)

4-Train/Test Split

In [57]:
ss=StandardScaler()
x_scaled=ss.fit_transform(x)
x_scaled=pd.DataFrame(x_scaled,columns=columns)

x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=0.2,random_state=42)

In [59]:
x_train_df=x_train.copy()
x_train_df['target']=y_train

In [ ]:
x_train

In [ ]:
data_melted=pd.melt(x_train_df,id_vars='target',var_name='features',value_name='value')
plt.figure(figsize=(20,20))
sns.boxplot(x='features',y='value',hue='target',data=data_melted)
plt.xticks(rotation=90)
plt.show()

5- K-NN Classification

In [ ]:
knn=KNeighborsClassifier(n_neighbors=2)
knn.fit(x_train,y_train)
y_pred=knn.predict(x_test)


In [ ]:
cm=confusion_matrix(y_test,y_pred)
print(cm)
sns.heatmap(cm,annot=True)
plt.show()

In [ ]:
print(accuracy_score(y_test,y_pred))

Finding Best Parameters

In [ ]:
k_range=list(range(1,16))
#uniform=all points' weights are equal, distance=negative correlation with distance

knn=KNeighborsClassifier()
grid=GridSearchCV(knn,param_grid=dict(n_neighbors=np.arange(1,16),weights=["uniform","distance"]),cv=10,scoring='accuracy')
grid.fit(x_train,y_train)
print(grid.best_score_)
print(grid.best_params_)

In [ ]:
knn=KNeighborsClassifier(n_neighbors=grid.best_params_['n_neighbors'],weights=grid.best_params_['weights'])

knn.fit(x_train,y_train)
y_pred=knn.predict(x_test)

cm=confusion_matrix(y_test,y_pred)
print(cm)
sns.heatmap(cm,annot=True)
plt.show()




In [ ]:
print(accuracy_score(y_test,y_pred))
